In [1]:
import pandas as pd
import numpy as np

#  차트 설정
%matplotlib inline
import matplotlib.pyplot as plt

### 함수

In [2]:
def 주별수익(data):
    result = data/data.shift(1)
    return result

def 절대모멘텀(data,month):
    # shift를 사용하여 원화는 달의 수익률을 구한다
    # 당월주가 > n 개월 전 주가(n개월 모멘텀 > 0) --> 주식매수
    # 당월주가 < n 개월 전 주가(n개월 모멘텀 < 0) --> 주식매도
    result = np.where(data/data.shift(month)>1,1,0)
    return result

def CAGR(data):
    y = data.index.year.unique()
    result = (data[-1]/data[0])**(1/len(y))-1
    return np.round(result,4)

def MDD(data):
    window = len(data)

    # Calculate the max drawdown in the past window days for each day in the series.
    # Use min_periods=1 if you want to let the first 252 days data have an expanding window
    Roll_Max = data.rolling(window, min_periods=1).max()
    Roll_Max.rename("Roll_Max", inplace=True)
    Drawdown = data/Roll_Max - 1.0
    Drawdown.rename("Drawdown", inplace=True)

    # Next we calculate the minimum (negative) daily drawdown in that window.
    # Again, use min_periods=1 if you want to allow the expanding window
    Max_Drawdown = Drawdown.rolling(window, min_periods=1).min()
    Max_Drawdown.rename("Max_Drawdown", inplace=True)
    
    return np.round(Drawdown,4), np.round(Max_Drawdown,4)

### 24주 평균모멘텀 구하기

In [3]:
def 주24평균모멘텀(data):
    # shift를 사용하여 원화는 달의 수익률을 구한다
    # 당월주가 > n 개월 전 주가(n개월 모멘텀 > 0) --> 주식매수
    # 당월주가 < n 개월 전 주가(n개월 모멘텀 < 0) --> 주식매도
    result = 0
    for i in range(1,25):
        result += np.where(data/data.shift(i)>1,1,0) 
    result = np.round(result/24,2)
    # 평균스코어 매매시 밴치마크 지수를 너무 쫓아가지 못함. 특정 점수 이상은 '1', 즉 전부 들어가는 방안에 고민이 필요함.
    # result[result>0.5] = 1
    return result

### Main

### 파일 읽어오기

In [4]:
mvc = pd.read_csv("밸류퀄리티모멘텀_종합순위.csv",sep=",",encoding="euc-kr",engine="python")

### total rank 우선순위 구하기

In [5]:
mvc.sort_values(["total_rank"],ascending=True,inplace=True)

In [6]:
import re
code = mvc["코드번호"].apply(lambda x:re.sub("A","",x))

### 종목 data 가져오기

In [7]:
close = pd.read_csv("20190126 KRX전체종목Colse가격Data.csv",sep=",",encoding="utf-8", engine="python")

In [8]:
close.index = pd.to_datetime(close["Date"])
del close["Date"]

In [9]:
data = pd.Series()
for i in range(0,100):
    cl = close[code[i]]
    data = pd.concat([data,cl], axis=1)    

In [10]:
data.head()

,0,005930,000660,068270,051910,005380,017670,207940,005490,015760,...,017800,016360,253450,008560,011780,000100,004170,007310,030000,010060
2010-01-04,NaN,16180,24100,15327,224000,119000,170000,NaN,612000,34250,...,43874,63247,NaN,1157,20850,147634,352531,149000,12476,215000
2010-01-05,NaN,16440,23350,15524,222500,110000,168500,NaN,618000,34050,...,44026,64290,NaN,1139,20650,148473,340057,148000,12275,213500
2010-01-06,NaN,16820,24550,15868,222500,111000,169000,NaN,612000,34000,...,43950,64764,NaN,1135,22000,150150,341370,150000,12135,216500
2010-01-07,NaN,16260,24400,15524,213000,106000,171500,NaN,606000,33800,...,44026,64290,NaN,1148,22400,148891,340714,148500,11994,218500
2010-01-08,NaN,16420,24650,15722,216000,106000,169500,NaN,606000,33800,...,43723,66566,NaN,1180,21550,147634,338088,151000,11673,227000


In [11]:
# 빈 컬럼 삭제
data.drop([0],axis=1, inplace=True)

### 구간 데이타 만들기

In [12]:
data = data["2018":]

In [13]:
### 매주금요일에 매매 들어감, pandas.core.groupby.SeriesGroupBy 객체가 만들어짐
Weekly_data = data.resample('W-FRI')

### 24주 평균모멘텀 값 구하기, CAGR/MDD 구하기

In [14]:
for icode in range(0,100):

    #실제 거래가 발생하는 자본금 = real_trading_자본금, system stop 그래프 = virtual_trading_자본금
    virtual_trading_자본금 = [1]
    system_stop = []
    매수금액 = 0
    매매금액 = 0
    #
    Qdata = Weekly_data[code[icode]]
    #  pandas.core.groupby.SeriesGroupBy 객체를 시리즈 객체로 만들기
    Qdata = Qdata.apply(lambda x:x) 
    Qdata = Qdata.dropna()
    Qdata = Qdata/Qdata[0]  #re-scale로 1로 맞춤
    
    # 24주 평균모멘텀 구하기
    score = 주24평균모멘텀(Qdata)
    #momentum score 시리즈 객체 만들기
    sscore = pd.Series(score, index=Qdata.index)
    ma20 = sscore.rolling(window=20,min_periods=1).mean()

    # 주별 수익구하기
    주수익 = 주별수익(Qdata)
    
    for v_score in range(0,len(score)):
        # 첫달 매수금액
        매수금액 = ma20[v_score] * virtual_trading_자본금[v_score]

        try:        
            #두번째주 수익과 첫주 매수금액 곱하기
            매매금액 = 매수금액 * 주수익[v_score+1]
            #첫달 자본금으로 두번째달 월별수익을 곱하여 손익을 계산하고 두번째달 자본금으로 할당함.
            virtual_trading_자본금.append((virtual_trading_자본금[v_score] - 매수금액) + 매매금액)  #현금 = 자본금 - 매수금액
        except IndexError:  #마지막달 매수금액의 투자수익은 구할수없어 pass 처리함
            pass

    virtual_trading_자본금 = pd.Series(virtual_trading_자본금, index=Qdata.index)

    system_stop = virtual_trading_자본금.rolling(window=24, min_periods=1).mean()
    #
    #
    #real trading 구하기
    ### 수익곡선 & 7개월 평균 모멘텀을 결합한 모멘텀 전략
    # 1.7개월 평균 모멘텀 스코어가 수익곡선 값보다 작으면 system stop
    # 2.7개월 평균 모멘텀 스코어가 수익곡선 값보다 크면 system go
    매수금액 = 0
    매매금액= 0
    real_trading_자본금 = [1]  #첫달 자본금 1로 설정

    for ireal in range(0,len(score)-1):
        # virtual_trading_자본금 > system_stop이면 매매거래
        if virtual_trading_자본금[ireal] > system_stop[ireal]:
            # 첫달 매수금액
            매수금액 = ma20[ireal] * real_trading_자본금[ireal]
            try:        
                #두번째달 수익과 첫달 매수금액 곱하기
                매매금액 = 매수금액 * 주수익[ireal+1]
                #첫달 자본금으로 두번째달 월별수익을 곱하여 손익을 계산하고 두번째달 자본금으로 할당함.
                real_trading_자본금.append((real_trading_자본금[ireal] - 매수금액) + 매매금액)  #현금 = 자본금 - 매수금액
            except IndexError:  #마지막달 매수금액의 투자수익은 구할수없어 pass 처리함
                pass
        else:
            real_trading_자본금.append(real_trading_자본금[ireal])

    real_trading_자본금 = pd.Series(real_trading_자본금, index=Qdata.index)
    
    Qdata_CAGR = CAGR(Qdata) # CAGR 구하기
    Qdata_DD, Qdata_MDD  = MDD(Qdata)  # MDD 구하기
    Qdata_누적주별수익 = 주별수익(Qdata).cumprod()  #누적수익률 구하기
    virtual_trading_CAGR = CAGR(virtual_trading_자본금) # CAGR 구하기
    virtual_trading_DD, virtual_trading_MDD = MDD(virtual_trading_자본금)  # MDD 구하기
    virtual_trading_누적주별수익 = 주별수익(virtual_trading_자본금).cumprod()  #누적수익률 구하기
    
    # 자본과 수익곡선 그래프 그리기 & 저장하기
    plt.rcParams["font.family"] = "NanumGothic"
    plt.rcParams['lines.linewidth'] = 1
    plt.rcParams["axes.grid"] = True
    
    textstr = '\n'.join((
    r"$Qdata_CAGR=%.2f$" % (Qdata_CAGR, ),
    r'$Qdata_MDD=%.2f$' % (Qdata_MDD[-1], ),
    r'$Qdata_earning=%.2f$' % (Qdata_누적주별수익[-1], ),
    r'$virtual_trading_CAGR=%.2f$' % (virtual_trading_CAGR, ),
    r'$virtual_trading_MDD=%.2f$' % (virtual_trading_MDD[-1], ),
    r'$virtual_trading_earning=%.2f$' % (virtual_trading_누적주별수익[-1], ) ))
    
    fig, axs = plt.subplots(2, 1,figsize=(14,6))
    axs[0].plot(Qdata,color="Black",label="Qdata")
    axs[0].plot(virtual_trading_자본금, color="red",label="24주평균모멘텀")
    axs[0].set_xlabel('Weekly')
    axs[0].set_ylabel('Close')
    axs[0].legend();
    
    axs[1].plot(sscore,color="Red", label="score")
    axs[1].plot(ma20,color="Blue", label="ma20")
    axs[1].set_ylabel('Momentum Score')
    
    
    # place a text box in upper left in axes coords
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    axs[0].text(0.05, 0.95, textstr, transform=axs[0].transAxes, fontsize=14, verticalalignment='top', bbox=props)

    # 그림으로 저장하기
    fig.savefig(code[icode]+".png")
    # plot에서 close하지 않으면 리소스가 과도하게 소모됨
    plt.cla()
    plt.clf()
    plt.close("all")